# Creating some ML pipelines

### Introduction

I thought I'd do a quick write up on how you can build some simple and effective ML pipelines using sklearn.

I found discovered the pipeline/gridsearch combo a few weeks ago after sending off some of my code to review.
In sending off my code I realized that were a few things that I had tweaked for performance, but weren't obvious to the reviewer.

- I had median imputed some variables (continuous features), while other variables were filled by mode
- I did a large amount of feature engineering, only to use a subset of those features in my model building (they were the best I swear)

So even though I did some work to get to that reviewed copy, these experiments that I went through during the process wouldn't be easy to understand unless the reviewer at all my revisions etc.

But then I found **pipelines / gridsearch** and all was good in the world.

**The pipelines let you combine all your feature engineering / pre-processing / modelling into one object**
**Gridsearch then lets you test all your assumptions / hyperparameters to find out which combinations generate the best result**

I haven't seen many write ups about them so I thought I'd do one myself.

References:

- http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
- http://scikit-learn.org/stable/modules/pipeline.html
- http://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
- https://stackoverflow.com/questions/33091376/python-what-is-exactly-sklearn-pipeline-pipeline
- http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html
- https://michelleful.github.io/code-blog/2015/06/20/pipelines/

# Libraries + MAE

In [12]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Imputer, PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import os
import time

import warnings
warnings.filterwarnings("ignore")

def MAE(y, ypred):
    
    import numpy as np
    
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)   

# Data read in and prep
### Making sure the data is in a lightGBM friendly format

In [3]:
basePath = 'C:/Users/Evan/Documents/GitHub/Zillow'
funcPath = 'C:/Users/Evan/Documents/GitHub/zillow_sklearn/Python'
subPath = 'F:/Nerdy Stuff/Kaggle submissions/Zillow'

os.chdir(basePath)

train = pd.read_csv('data/train_2016_v2.csv')
properties = pd.read_csv('data/properties_2016.csv', low_memory=False)
sample = (pd.read_csv('data/sample_submission.csv')
            .rename(columns = {'ParcelId':'parcelid'}))

for c, dtype in zip(properties.columns, properties.dtypes):	
    if dtype == np.float64:
        properties[c] = properties[c].astype(np.float32)

df_train = (train.merge(properties, how='left', on='parcelid')
            .drop(['parcelid', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1))

train_columns = df_train.columns 

# Splitting the dataset to train/valid sets

In [4]:
valid = df_train.iloc[1:20000, :]
train = df_train.iloc[20001:90275, :]

y_train = train['logerror'].values
y_valid = valid['logerror'].values

x_train = train.drop('logerror', axis = 1)
x_valid = valid.drop('logerror', axis = 1)

idVars = [i for e in ['id',  'flag', 'has'] for i in list(train_columns) if e in i] + ['fips', 'hashottuborspa']
countVars = [i for e in ['cnt',  'year', 'nbr', 'number'] for i in list(train_columns) if e in i]
taxVars = [col for col in train_columns if 'tax' in col and 'flag' not in col]
          
ttlVars = idVars + countVars + taxVars
dropVars = [i for e in ['census',  'tude', 'error'] for i in list(train_columns) if e in i]
contVars = [col for col in train_columns if col not in ttlVars + dropVars]

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)
    
for c in x_valid.dtypes[x_valid.dtypes == object].index.values:
    x_valid[c] = (x_valid[c] == True)   

# The first pipeline

Since everyone is using lightGBM I'll use that. 
Initially we'll just look at the continuous variables in model building, but we'll extend that out too.

So let's start with the easy pipeline that:

- Imputes the missing values with the median
- Selects the best 5 features
- Builds a LightGBM model

In [5]:
print(contVars)

x_train_cont = x_train[contVars]
x_valid_cont = x_valid[contVars]

['basementsqft', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'garagetotalsqft', 'lotsizesquarefeet', 'poolsizesum', 'yardbuildingsqft17', 'yardbuildingsqft26']


In [13]:
pipeline = Pipeline(
                    [('imp', Imputer(missing_values='NaN', strategy = 'median', axis=0)),
                     ('feat_select', SelectKBest(k = 5)),
                     ('lgbm', LGBMRegressor())
                     
])

pipeline.fit(x_train_cont, y_train)   

y_pred = pipeline.predict(x_valid_cont)
print('MAE on validation set: %s' % (round(MAE(y_valid, y_pred), 5)))

MAE on validation set: 0.0735


## Pipeline 2.0 - oh hai there gridsearch

But from the above code we have made a few assumptions that haven't been tested.

**We assume that:**
- Median is the best way of imputing the variables
- Only 5 variables needed for the lowest error 

But we don't need to assume these, we can test these assumptions and find out which actually results in the lowest error.

In [17]:
pipeline = Pipeline(
                    [('imp', Imputer(missing_values='NaN', axis=0)),
                     ('feat_select', SelectKBest()),
                     ('lgbm', LGBMRegressor())
                     
])

parameters = {}
parameters['imp__strategy'] = ['mean', 'median', 'most_frequent']
parameters['feat_select__k'] = [5, 10]

CV = GridSearchCV(pipeline, parameters, scoring = 'mean_absolute_error', n_jobs= 1)
CV.fit(x_train_cont, y_train)   

print('Best score and parameter combination = ')

print(CV.best_score_)    
print(CV.best_params_)    

y_pred = CV.predict(x_valid_cont)
print('MAE on validation set: %s' % (round(MAE(y_valid, y_pred), 5)))

Best score and parameter combination = 
-0.0669848314136
{'feat_select__k': 5, 'imp__strategy': 'most_frequent'}
MAE on validation set: 0.0735


Interesting, I never thought that using a mode would come out on top.

But we since we're also here we let's also test and see what is the best imputation policy for tax variables. First I'll quickly write a column extractor that plays nicely with the pipeline.

These can look hard at first, but they definitely get easier as you write a few. 
And since we're writing some code we should probably do some testing to make sure that it works the way we think it will.

# Column extractor

## Takes a list of columns and returns a df with those cols

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, subset):
        self.subset = subset

    def transform(self, X, *_):
        return X.loc[:, self.subset]

    def fit(self, *_):
        return self

# Testing

Since we've already created x_train_cont I'll test the column extractor on this case

In [24]:
contExtractor = ColumnExtractor(contVars)
x_train_cont_test = contExtractor.transform(x_train).head()

x_train_cont.head().equals(x_train_cont_test)

True

# Pipeline 3.0 - taxes

It might be worth while to outline this pipeline before we proceed


In [ ]:
pipeline = Pipeline([
                    ('tax', ColumnExtractor(taxVars)),
                    ('imp', Imputer(missing_values='NaN', axis=0)),
                    ('feat_select', SelectKBest()),
                    ('lgbm', LGBMRegressor())
                     
])

parameters = dict(imp__strategy=['mean', 'median', 'most_frequent'],
                    feat_select__k=[5, 2, 1] 

)   

CV = GridSearchCV(pipeline, parameters, scoring = 'neg_mean_absolute_error', n_jobs= 1)
CV.fit(x_train, y_train)   

print(CV.best_params_)    
print(CV.best_score_)    

y_pred = CV.predict(x_valid)
print('MAE on validation set: %s' % (round(MAE(y_valid, y_pred), 5)))

In [31]:
pipeline = Pipeline([
        ('union', FeatureUnion([
            ('continuous', Pipeline([
                    ('contExtract', ColumnExtractor(contVars)),
                    ('imp', Imputer(missing_values='NaN', axis=0)),
                    ('feats', FeatureUnion([
                                 ('feat2', PolynomialFeatures(2)),
                                 ('pca5', PCA(n_components= 5)),
                                 ('pca10', PCA(n_components= 10))
                                 ])),
                    ('scaler', StandardScaler()),
                    ])
            ), 
            ('taxVars', Pipeline([
                    ('taxExtract', ColumnExtractor(taxVars)),
                    ('feats', FeatureUnion([
                                 ('feat2', PolynomialFeatures(2)),
                                 ('pca5', PCA(n_components= 5)),
                                 ('pca10', PCA(n_components= 10))
                                 ]))
                    ]))                
                ])),
    ('feat_select', SelectKBest()),
    ('lgbm', LGBMRegressor())             
])

{'feat_select__k': 5, 'imp__strategy': 'most_frequent'}
-0.0762164915336
MAE on validation set: 0.07704


In [36]:
len(taxVars)

5

In [41]:
pipeline = Pipeline([
        ('union', FeatureUnion([
            ('continuous', Pipeline([
                    ('contExtract', ColumnExtractor(contVars)),
                    ('cont_imp', Imputer(missing_values='NaN', axis=0)),
                    ('feats', FeatureUnion([
                                 ('feat2', PolynomialFeatures(2)),
                                 ('pca5', PCA(n_components= 5)),
                                 ('pca10', PCA(n_components= 10))
                                 ])),
                    ('scaler', StandardScaler()),
                    ])
            ), 
        ])),
    ('feat_select', SelectKBest()),
    ('rf', RandomForestRegressor())              
])

parameters = {}

parameters['cont_imp__strategy'] = ['mean', 'median', 'most_frequent']
parameters['feat_select__k'] = [5, 10, 25]

CV = GridSearchCV(pipeline, parameters, scoring = 'neg_mean_absolute_error', n_jobs= 1)
CV.fit(x_train, y_train)   

#print(CV.best_params_)    
#print(CV.best_score_)    

#y_pred = CV.predict(x_valid)
#print('MAE on validation set: %s' % (round(MAE(y_valid, y_pred), 5)))

ValueError: Invalid parameter cont_imp for estimator Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('continuous', Pipeline(steps=[('contExtract', ColumnExtractor(subset=['logerror', 'basementsqft', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'fin...timators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]). Check the list of available parameters with `estimator.get_params().keys()`.